In [2]:
from ase.atoms import Atoms
from ase.io import write
from ase.visualize import view
import jax
import jax.numpy as jnp
import sys
sys.path.append('..')
import analysis
import datatypes
from input_pipeline import ase_atoms_to_jraph_graph, generate_fragments
from qm9 import load_qm9, read_sdf
from train import evaluate_step, get_predictions

2023-03-01 23:57:05.577671: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib:/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/pkgs/cudatoolkit-11.3.1-h2bc3f7f_2/lib
2023-03-01 23:57:05.583039: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib:/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/pkgs/cudatoolkit-11.3.1-h2bc3f7f_2/lib
2023-03-01 23:57:05.583050: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia

In [3]:
config, best_state, metrics_for_best_state = analysis.load_from_workdir('../workdirs/mace1')

/home/gridsan/sekim/spherical-harmonic-net/qm9_data_tf/
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-03-01 23:57:28.387736: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib:/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/pkgs/cudatoolkit-11.3.1-h2bc3f7f_2/lib
2023-03-01 23:57:28.387764: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
qm9_data = list(read_sdf(open('/home/gridsan/sekim/spherical-harmonic-net/qm9_data/gdb9.sdf', 'r')))
atoms = qm9_data[1]

In [5]:
atomic_numbers = jnp.array([1, 6, 7, 8, 9])
numbers_to_symbols = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 9: 'F'}

def get_numbers(species: jnp.ndarray):
    numbers = []
    for i in species:
        numbers.append(atomic_numbers[i])
    return jnp.array(numbers)

In [6]:
cutoff = 5.0
key = jax.random.PRNGKey(0)
epsilon = 1e-4

graph = ase_atoms_to_jraph_graph(atoms, atomic_numbers, cutoff)
frags = generate_fragments(key, graph, len(atomic_numbers), epsilon)
frags = list(frags)

In [9]:
type(frags[1])

jraph._src.graph.GraphsTuple

In [7]:
preds = get_predictions(best_state, frags[1])

TracerArrayConversionError: The numpy.ndarray conversion method __array__() was called on the JAX Tracer object Traced<ShapedArray(int32[])>with<BatchTrace(level=1/0)> with
  val = Array([2, 0], dtype=int32)
  batch_dim = 0
This BatchTracer with object id 139804409253216 was created on line:
  /home/gridsan/sekim/.local/lib/python3.9/site-packages/haiku/_src/stateful.py:833 (mapped_fun)
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerArrayConversionError

In [ ]:
preds

In [ ]:
frag = frags[3]
frag_atoms = Atoms(positions=frag.nodes.positions, numbers=get_numbers(frag.nodes.species))
view(frag_atoms, viewer='x3d')

In [ ]:
# save to image files instead:
for i, frag_to_write in enumerate(frags):
    write(f'image{i}.png', Atoms(positions=frag_to_write.nodes.positions, numbers=get_numbers(frag_to_write.nodes.species)))